In [9]:
library(tidyverse)
library(tidymodels)
library(repr)
library(dplyr)
options(repr.matrix.max.rows = 6)

In [10]:
heart_data <- read_csv("heart_data.csv") |>
select(chol, trestbps, fbs, age, sex, num) |>
mutate(sex = as_factor(sex))|>
mutate(sex = fct_recode(sex, "male" = "1", "female" = "0")) |>
mutate(fbs = as_factor(fbs)) |>
mutate(fbs = fct_recode(fbs, "true" = "1", "false" = "0")) |>
rename("fbs_gt_120" = "fbs")
heart_data

Rows: 303 Columns: 14
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
dbl (14): age, sex, cp, trestbps, chol, fbs, restecg, thalach, exang, oldpea...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


chol,trestbps,fbs_gt_120,age,sex,num
<dbl>,<dbl>,<fct>,<dbl>,<fct>,<dbl>
233,145,true,63,male,0
286,160,false,67,male,2
229,120,false,67,male,1
⋮,⋮,⋮,⋮,⋮,⋮
131,130,false,57,male,3
236,130,false,57,female,1
175,138,false,38,male,0
